In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


# Initialize Spark Session with MySQL JDBC Driver
spark = SparkSession.builder.master('local[2]').appName('dataframe-Tranform-demo').config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33").getOrCreate()

#df = spark.read.csv('s3://feb2025-training-bucket/data/5m-Sales-Records.csv', header = True)
#df = spark.read.csv('/mnt/s3_feb2025/data/5m-Sales-Records.csv', header = True)
#df = spark.read.csv('abfss://4afe1d84-eaf5-405f-a28b-03b576203110@onelake.dfs.fabric.microsoft.com/bb6f7778-293e-4c55-8576-e25b56eddb04/Files/5m-Sales-Records.csv', header = True)


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d3643721-3d7f-46ee-ba9f-e016e5d25938;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
:: resolution report :: resolve 203ms :: artifacts dl 7ms
	:: modules in use:
	com.google.protobuf#protobuf-java;3.21.9 from central in [default]
	com.mysql#mysql-connector-j;8.0.33 from central in [default]
	mysql#mysql-connector-java;8.0.33 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------------------------------------

In [3]:
df = spark.read.csv('data/5m-Sales-Records.csv', header = True)
df.printSchema()
df.show(5)

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: string (nullable = true)
 |-- Unit Price: string (nullable = true)
 |-- Unit Cost: string (nullable = true)
 |-- Total Revenue: string (nullable = true)
 |-- Total Cost: string (nullable = true)
 |-- Total Profit: string (nullable = true)

+--------------------+-------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+-------+---------------+------

In [4]:
for col_name in df.columns:
    new_col_name = col_name.replace(" ", "_")
    df = df.withColumnRenamed(col_name, new_col_name)
df.createOrReplaceTempView("salesdata")
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Priority: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: string (nullable = true)
 |-- Unit_Price: string (nullable = true)
 |-- Unit_Cost: string (nullable = true)
 |-- Total_Revenue: string (nullable = true)
 |-- Total_Cost: string (nullable = true)
 |-- Total_Profit: string (nullable = true)



#### 1. Casting Data Types

In [5]:


# Cast columns to the desired data types
df_casted = df \
    .withColumn("Order_ID", F.col("Order_ID").cast("int")) \
    .withColumn("Units_Sold", F.col("Units_Sold").cast("int")) \
    .withColumn("Unit_Price", F.col("Unit_Price").cast("double")) \
    .withColumn("Unit_Cost", F.col("Unit_Cost").cast("double")) \
    .withColumn("Total_Revenue", F.col("Total_Revenue").cast("double")) \
    .withColumn("Total_Cost", F.col("Total_Cost").cast("double")) \
    .withColumn("Total_Profit", F.col("Total_Profit").cast("double"))

# Show the updated DataFrame
df_casted.show()

# Print the schema to verify the changes
df_casted.printSchema()


+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|       Palau|Office Supplies|       Online|             H|  3/6/2016|517073523| 3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25|
|              Europe|      Poland|      Beverages|       Online|             L| 4/18/2010|380507028| 5/26/2010|      9340|     47.45|    31.79|     443183.0|  296918.6|    146264.4|
|       North America|      Canada|         Cereal|       Online|             M|  1/8

#### 1. Select Specific Columns

In [6]:

df.select("Region", "Country", F.date_format("Order_Date", "dd-MM-yyyy").alias("Order_Date")).show()

+--------------------+------------+----------+
|              Region|     Country|Order_Date|
+--------------------+------------+----------+
|Australia and Oce...|       Palau|      null|
|              Europe|      Poland|      null|
|       North America|      Canada|      null|
|              Europe|     Belarus|      null|
|Middle East and N...|        Oman|      null|
|  Sub-Saharan Africa|Burkina Faso|      null|
|              Europe|  Montenegro|      null|
|Middle East and N...|  Azerbaijan|      null|
|  Sub-Saharan Africa| South Sudan|      null|
|       North America|   Greenland|      null|
|              Europe|    Portugal|      null|
|  Sub-Saharan Africa|Sierra Leone|      null|
|              Europe|     Germany|      null|
|              Europe|  Montenegro|      null|
|                Asia|  Bangladesh|      null|
|  Sub-Saharan Africa|  Mauritius |      null|
|                Asia|  Kyrgyzstan|      null|
|  Sub-Saharan Africa|  Madagascar|      null|
|            

#### 2. Filter Rows (Condition-Based)

In [7]:
df.filter(df["Country"] == "Australia").limit(5).show()

+--------------------+---------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|              Region|  Country|    Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|
+--------------------+---------+-------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|Australia|    Beverages|      Offline|             C|12/14/2018|316851985| 1/3/2019|      2398|     47.45|    31.79|    113785.10|  76232.42|    37552.68|
|Australia and Oce...|Australia|         Meat|      Offline|             L| 1/31/2019|611135375|2/16/2019|      7393|    421.89|   364.69|   3119032.77|2696153.17|   422879.60|
|Australia and Oce...|Australia|      Clothes|      Offline|             L| 5/29/2015|734939706| 6/2/2015|      384

#### 3. Adding New Columns

In [8]:


df = df.withColumn("Status", F.lit(0)).withColumn("currentDate",F.lit( F.current_date()))

#df.printSchema()
df.show(2)

+--------------------+-------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+------+-----------+
|              Region|Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID|Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|Status|currentDate|
+--------------------+-------+---------------+-------------+--------------+----------+---------+---------+----------+----------+---------+-------------+----------+------------+------+-----------+
|Australia and Oce...|  Palau|Office Supplies|       Online|             H|  3/6/2016|517073523|3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25|     0| 2025-03-06|
|              Europe| Poland|      Beverages|       Online|             L| 4/18/2010|380507028|5/26/2010|      9340|     47.45|    31.79|    443183.00| 296918.60|   146264.40|     0| 2025-03-06|
+-------------------

In [ ]:
df = df.withColumn("Profit_Per_Unit", df["Total_Profit"] / df["Units_Sold"])
df.printSchema()

#### 4. Renaming Columns

In [9]:
dfnew = df.withColumnRenamed("Total_Revenue", "Revenue")
dfnew.show()

+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+----------+----------+------------+------+-----------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Priority|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|   Revenue|Total_Cost|Total_Profit|Status|currentDate|
+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+----------+----------+------------+------+-----------+
|Australia and Oce...|       Palau|Office Supplies|       Online|             H|  3/6/2016|517073523| 3/26/2016|      2401|    651.21|   524.96|1563555.21|1260428.96|   303126.25|     0| 2025-03-06|
|              Europe|      Poland|      Beverages|       Online|             L| 4/18/2010|380507028| 5/26/2010|      9340|     47.45|    31.79| 443183.00| 296918.60|   146264.40|     0| 2025-03-06|
|    

#### 5. Dropping Columns

In [11]:
df = df.drop("Order_Priority").drop("Status")
df.show()

+--------------------+------------+---------------+-------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|currentDate|
+--------------------+------------+---------------+-------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|Australia and Oce...|       Palau|Office Supplies|       Online|  3/6/2016|517073523| 3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25| 2025-03-06|
|              Europe|      Poland|      Beverages|       Online| 4/18/2010|380507028| 5/26/2010|      9340|     47.45|    31.79|    443183.00| 296918.60|   146264.40| 2025-03-06|
|       North America|      Canada|         Cereal|       Online|  1/8/2015|504055583| 1/31/2015|   

#### 6. Handling Missing Data

In [12]:
df_cleaned = df.na.drop()
df_cleaned.count()



5000000

In [13]:
df_filled = df.na.fill({"Total_Revenue": 0})
df_filled.show()

+--------------------+------------+---------------+-------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|              Region|     Country|      Item_Type|Sales_Channel|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|currentDate|
+--------------------+------------+---------------+-------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|Australia and Oce...|       Palau|Office Supplies|       Online|  3/6/2016|517073523| 3/26/2016|      2401|    651.21|   524.96|   1563555.21|1260428.96|   303126.25| 2025-03-06|
|              Europe|      Poland|      Beverages|       Online| 4/18/2010|380507028| 5/26/2010|      9340|     47.45|    31.79|    443183.00| 296918.60|   146264.40| 2025-03-06|
|       North America|      Canada|         Cereal|       Online|  1/8/2015|504055583| 1/31/2015|   

#### 7. Aggregation and Grouping

In [27]:

df= df.withColumn("Total_Revenue", F.col("Total_Revenue").cast("double"))
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Ship_Date: string (nullable = true)
 |-- Units_Sold: string (nullable = true)
 |-- Unit_Price: string (nullable = true)
 |-- Unit_Cost: string (nullable = true)
 |-- Total_Revenue: double (nullable = true)
 |-- Total_Cost: string (nullable = true)
 |-- Total_Profit: string (nullable = true)
 |-- currentDate: date (nullable = false)



In [28]:

df.groupBy("Country").agg(F.sum("Total_Revenue").alias("Total_Revenue_by_Country")).show()

+--------------------+------------------------+
|             Country|Total_Revenue_by_Country|
+--------------------+------------------------+
|              Turkey|         3.6078570441E10|
|              Angola|         3.5938680403E10|
|          East Timor|          3.606126212E10|
|             Albania|         3.5926849662E10|
|           Nicaragua|         3.5761324107E10|
|               China|         3.6012702269E10|
|             Somalia|         3.5987560348E10|
|             Croatia|         3.6079982499E10|
|             Andorra|         3.5973797336E10|
|              Norway|         3.5834656122E10|
|               Niger|         3.5985414009E10|
|             Ireland|          3.607508131E10|
|       Liechtenstein|         3.6084261206E10|
|           Swaziland|         3.5806411272E10|
|Democratic Republ...|         3.5890804111E10|
|              Bhutan|         3.6090325133E10|
|              Israel|         3.6018502809E10|
|            Tunisia |          3.581874

#### 8. Sorting

In [29]:
df.orderBy(df["Total_Revenue"].desc()).show()

+--------------------+--------------------+---------+-------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|              Region|             Country|Item_Type|Sales_Channel|Order_Date| Order_ID| Ship_Date|Units_Sold|Unit_Price|Unit_Cost|Total_Revenue|Total_Cost|Total_Profit|currentDate|
+--------------------+--------------------+---------+-------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+-----------+
|              Europe|              Sweden|Household|      Offline| 1/23/2012|668456542|  3/6/2012|     10000|    668.27|   502.54|    6682700.0|5025400.00|  1657300.00| 2025-03-06|
|              Europe|              Sweden|Household|      Offline| 1/23/2012|668456542|  3/6/2012|     10000|    668.27|   502.54|    6682700.0|5025400.00|  1657300.00| 2025-03-06|
|                Asia|          Kyrgyzstan|Household|       Online| 8/28/2011|336815345| 9

#### 9.Distinct Values

In [30]:
df.select("country").distinct().count()

185

#### 10. Joining DataFrames

In [31]:
customers = spark.read.csv('data/customers.csv', header = True, inferSchema = True)
customerorder = spark.read.csv('data/customerorder_04102024.csv', header = True, inferSchema = True)
customerorder.printSchema()


root
 |-- Order_ID: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [33]:

join1 = df.join(customerorder, ["Order_ID","Order_ID"], how="inner")
final_join = join1.join(customers, on="customer_id", how="inner")
result_df = final_join.select("Order_ID", "customer_id", "customer_name")
result_df.show()

+---------+-----------+--------------------+
| Order_ID|customer_id|       customer_name|
+---------+-----------+--------------------+
|669165933|   30585978|STEPHENS,  GERALD...|
|669165933|   11123757|     SMITH,  SHIRLEY|
|341417157|     349822|     GUZMAN,  CARMEN|
|514321792|   27652636| HASSETT,  PATRICK J|
|115456712|   14437343|     HENTZ,  DIANA L|
|547995746|   20441596|    TIRADO,  MARCO A|
|135425221|    5945686|     SKORA,  BRIAN S|
|871543967|    5385771|     SLAWEK,  DEAN J|
|770463311|    1427940|   REAVES,  LIONEL C|
|616607081|   10457387|BONGIOVANNI,  KEL...|
|814711606|   19154815|   VERTA,  RAYMOND J|
|939825713|    2469887|  BRYANT,  KATHRYN I|
|187310731|   11743397|      UCZEN,  MARK D|
|522840487|    9944827|  HOFFMANN,  AARON G|
|832401311|   11928169|      OUTTEN,  MIA G|
|972292029|   24917207| AGUILAR JR,  ROBERT|
|419123971|   17211232|    BALASZEK,  LES J|
|519820964|    7164613|  BERNARDINO,  JAIME|
|441619336|   23608911|KLIMASZEWSKI,  GR...|
|322067916

#### 11. Pivoting

In [ ]:
%sql
SELECT Country,Item_Type,
       SUM(CASE WHEN Item_Type = 'Office Supplies' THEN Total_Revenue ELSE 0 END) AS Office_Supplies_Revenue,
       SUM(CASE WHEN Item_Type = 'Beverages' THEN Total_Revenue ELSE 0 END) AS Beverages_Revenue
FROM salesdata
GROUP BY Country,Item_Type;

In [34]:
from pyspark.sql.functions import sum, when

result_df = (
    df.groupBy("Country", "Item_Type").agg(
        sum(when(df.Item_Type == 'Office Supplies', df.Total_Revenue).otherwise(0)).alias('Office_Supplies_Revenue'),
        sum(when(df.Item_Type == 'Beverages', df.Total_Revenue).otherwise(0)).alias('Beverages_Revenue')
    )
)
result_df.show()

+--------------------+---------------+-----------------------+-----------------+
|             Country|      Item_Type|Office_Supplies_Revenue|Beverages_Revenue|
+--------------------+---------------+-----------------------+-----------------+
|           Swaziland|      Beverages|                    0.0|     5.31458396E8|
|         South Korea|  Personal Care|                    0.0|              0.0|
|         Afghanistan|      Cosmetics|                    0.0|              0.0|
|              Panama|      Household|                    0.0|              0.0|
|            Barbados|      Baby Food|                    0.0|              0.0|
|           Macedonia|         Snacks|                    0.0|              0.0|
|Saint Vincent and...|        Clothes|                    0.0|              0.0|
|             Germany|         Fruits|                    0.0|              0.0|
|       Liechtenstein|      Cosmetics|                    0.0|              0.0|
|                Togo|      

#### 12. Window Functions

In [ ]:
%sql
SELECT Country,Order_Id,Total_Revenue,Units_Sold,
       RANK() OVER ( ORDER BY Total_Revenue DESC, Units_Sold DESC) AS revenue_units_rank
FROM salesdata;

In [35]:
from pyspark.sql import Window
from pyspark.sql.functions import col,rank

# window specification for ranking based on Total_Revenue and Units_Sold
window_spec = Window.orderBy(col("Total_Revenue").desc(), col("Units_Sold").desc())

ranked_df = df.withColumn("revenue_units_rank", rank().over(window_spec))
result_df = ranked_df.select("Country", "Order_ID", "Total_Revenue", "Units_Sold", "revenue_units_rank")
result_df.show()

25/03/06 04:50:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/06 04:50:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/06 04:50:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/06 04:50:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/03/06 04:50:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+---------+-------------+----------+------------------+
|    Country| Order_ID|Total_Revenue|Units_Sold|revenue_units_rank|
+-----------+---------+-------------+----------+------------------+
|      Yemen|214452111|    6682700.0|     10000|                 1|
|    Armenia|502635943|    6682700.0|     10000|                 1|
|   Zimbabwe|423730075|    6682700.0|     10000|                 1|
|   Slovakia|380272710|    6682700.0|     10000|                 1|
|    Comoros|546093308|    6682700.0|     10000|                 1|
|    Romania|992088878|    6682700.0|     10000|                 1|
|    Armenia|502635943|    6682700.0|     10000|                 1|
|    Comoros|546093308|    6682700.0|     10000|                 1|
|     Taiwan|301366841|    6682700.0|     10000|                 1|
|     Taiwan|301366841|    6682700.0|     10000|                 1|
|   Tanzania|135546243|    6682700.0|     10000|                 1|
| Cape Verde|257909476|    6682700.0|     10000|

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import col,rank,dense_rank

# window specification for ranking based on Total_Revenue and Units_Sold
window_spec = Window.orderBy(col("Total_Revenue").desc(), col("Units_Sold").desc())

ranked_df = df.withColumn("revenue_units_rank", dense_rank().over(window_spec))
result_df = ranked_df.select("Country", "Order_ID", "Total_Revenue", "Units_Sold", "revenue_units_rank")
result_df.show()

####  Chaining Transformations

In [ ]:
df_cleaned = (
    df.filter(df["Sales_Channel"] == "Online")  # Filter online sales
      .withColumn("Profit_Per_Unit", df["Total_Profit"] / df["Units_Sold"])  # Add new column
      .orderBy("Total_Revenue", ascending=False)  # Sort by total revenue
)

df_cleaned.show()

In [ ]:
df_cleaned = (
    df.filter(df["Sales_Channel"] == "Online")  # Filter online sales
      .withColumn("Profit_Per_Unit", df["Total_Profit"] / df["Units_Sold"])  # Add new column
      .groupBy("Country").agg(F.sum("Total_Revenue").alias("Total_Revenue_by_Country"))  # Aggregate revenue by region
      .orderBy("Total_Revenue_by_Country", ascending=False)  # Sort by total revenue
)

df_cleaned.show()

### Data Quality Checks

In [ ]:
from pyspark.sql import functions as F


# Apply the necessary transformations to create the error_flag column
df_with_error_flags = (
    spark.table("salesdata")
    .withColumn(
        "error_flag",
        F.when(F.col("Order_ID").isNull(), "Order_ID is NULL")
        .when(F.col("Country").isNull(), "Country is NULL")
        .when(F.col("Units_Sold").isNull(), "Units_Sold is NULL")
        .when(F.col("Units_Sold") < 0, "Units_Sold is negative")
        .when(F.col("Total_Revenue") < 0, "Total_Revenue is negative")
        .when(F.col("Total_Profit") < 0, "Total_Profit is negative")
        .when(F.col("Order_Date").isNull(), "Order_Date is NULL")
        .when(F.col("Ship_Date").isNull(), "Ship_Date is NULL")
        .when(
            ~F.col("Order_Date").rlike("^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$"),
            "Order_Date has invalid format (expected M/d/yyyy)"
        )
        .when(
            ~F.col("Ship_Date").rlike("^[1-9]{1}[0-9]{0,1}/[1-9]{1}[0-9]{0,1}/[0-9]{4}$"),
            "Ship_Date has invalid format (expected M/d/yyyy)"
        )
        # Uncomment this line if you want to include Ship_Date before Order_Date check
        # .when(F.col("Ship_Date") < F.col("Order_Date"), "Ship_Date before Order_Date")
        .otherwise(None)
    )
)

# Filter out records with error_flag as None (valid records)
valid_records_df = df_with_error_flags.filter(F.col("error_flag").isNull())

# Show the results
valid_records_df.show()

# Create a temporary view for further SQL operations
valid_records_df.createOrReplaceTempView("valid_salesdata")

# Filter error records where error_flag is NOT NULL
error_records_df = sales_df_with_flag.filter(F.col("error_flag").isNotNull())

# Show or process error records
error_records_df.show()

vaild_records_df = sales_df_with_flag.filter(F.col("error_flag").isNull())
print(vaild_records_df.count())

In [ ]:
# Aggregating data with groupBy and agg
salesdata_aggregated_df = (
    valid_records_df
    .groupBy("Region", "Country", "Item_Type", "Sales_Channel", "Order_Priority")
    .agg(
        F.sum("Units_Sold").alias("Total_Units_Sold"),
        F.sum("Total_Revenue").alias("Total_Revenue"),
        F.sum("Total_Cost").alias("Total_Cost"),
        F.sum("Total_Profit").alias("Total_Profit")
    )
)

In [ ]:
MYSQL_URL = "jdbc:mysql://172.17.0.3:3306/demo"
MYSQL_PROPERTIES = {
    'user': 'root',
    'password': '*7E567C9DC06217268D72D52BABCA14EAB8993ACF',
    'driver': 'com.mysql.jdbc.Driver',
    'allowPublicKeyRetrieval':'true',
    'useSSL':'false'
}
salesdata_aggregated_df.write.jdbc(url=MYSQL_URL, table="salesdata", mode="append", properties=MYSQL_PROPERTIES)